<a href="https://colab.research.google.com/github/EungyuHan/23_1_Database/blob/main/OverSampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [16]:
data = pd.read_csv('/content/drive/MyDrive/HR_Analytics_preprocessing.csv')

In [17]:
data

,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,Gender,HourlyRate,JobInvolvement,JobLevel,...,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Age_cut,FarFromHome,LowMonthlyIncome,LowWorkingYears,TotalSatisfaction
0,Yes,Travel_Rarely,1102,1,College,Life Sciences,Female,94,High,2,...,Bad,6,4,0,5,2,0,0,0,4
1,No,Travel_Frequently,279,8,Below College,Life Sciences,Male,61,Medium,2,...,Better,10,7,1,7,3,0,0,0,6
2,Yes,Travel_Rarely,1373,2,College,Other,Male,92,Medium,1,...,Better,0,0,0,0,2,0,1,0,6
3,No,Travel_Frequently,1392,3,Master,Life Sciences,Female,56,High,1,...,Better,8,7,3,0,1,0,1,0,7
4,No,Travel_Rarely,591,2,Below College,Medical,Male,40,High,1,...,Better,2,2,2,2,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,No,Travel_Frequently,884,23,College,Medical,Male,41,Very High,2,...,Better,5,2,0,3,2,1,1,0,7
1357,No,Travel_Rarely,613,6,Below College,Medical,Male,42,Medium,3,...,Better,7,7,1,7,2,0,0,0,3
1358,No,Travel_Rarely,155,4,Bachelor,Life Sciences,Male,87,Very High,2,...,Better,6,2,0,3,1,0,0,0,3
1359,No,Travel_Frequently,1023,2,Bachelor,Medical,Male,63,Medium,2,...,Good,9,6,0,8,3,0,0,0,7


In [18]:
Y = data['Attrition']
data.drop('Attrition', axis = 1, inplace=True)
X = data.copy()

In [22]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
label_encoder =  LabelEncoder()

categorical_features = X.select_dtypes(include=['object']).columns

# 범주형 데이터에 대해 레이블 인코딩 수행
label_encoder = LabelEncoder()
for feature in categorical_features:
    X[feature] = label_encoder.fit_transform(X[feature])

# 결과 확인
X.head()

,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,...,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Age_cut,FarFromHome,LowMonthlyIncome,LowWorkingYears,TotalSatisfaction
0,2,1102,1,2,1,0,94,0,2,7,...,0,6,4,0,5,2,0,0,0,4
1,1,279,8,1,1,1,61,2,2,6,...,2,10,7,1,7,3,0,0,0,6
2,2,1373,2,2,4,1,92,2,1,2,...,2,0,0,0,0,2,0,1,0,6
3,1,1392,3,4,1,0,56,0,1,6,...,2,8,7,3,0,1,0,1,0,7
4,2,591,2,1,3,1,40,0,1,2,...,2,2,2,2,2,1,0,0,0,4


In [44]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   BusinessTravel           1361 non-null   int64
 1   DailyRate                1361 non-null   int64
 2   DistanceFromHome         1361 non-null   int64
 3   Education                1361 non-null   int64
 4   EducationField           1361 non-null   int64
 5   Gender                   1361 non-null   int64
 6   HourlyRate               1361 non-null   int64
 7   JobInvolvement           1361 non-null   int64
 8   JobLevel                 1361 non-null   int64
 9   JobRole                  1361 non-null   int64
 10  MaritalStatus            1361 non-null   int64
 11  MonthlyRate              1361 non-null   int64
 12  NumCompaniesWorked       1361 non-null   int64
 13  OverTime                 1361 non-null   int64
 14  PercentSalaryHike        1361 non-null   int64
 15  Perf

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, random_state=42)

In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler

# 랜덤포레스트 분류기 객체 생성
oversampler = RandomOverSampler()
rf = RandomForestClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [80]:
param_grid = {
    'n_estimators': range(100,501),
    'max_depth': list(range(0,15)) + [None],
    'min_samples_split': range(1,50),
    'min_samples_leaf': range(1,50),
}

In [81]:
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=100,
    scoring='accuracy',
    cv=5,
    random_state=42
)
random_search.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   param_distributions={'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14,
                                                      None],
                                        'min_samples_leaf': range(1, 50),
                                        'min_samples_split': range(1, 50),
                                        'n_estimators': range(100, 501)},
                   random_state=42, scoring='accuracy')

In [84]:
best_model = random_search.best_estimator_
best_params = random_search.best_params_
print("Best Model:", best_model)
print("Best Parameters:", best_params)

test_accuracy = best_model.score(X_test, Y_test)
print("Test Accuracy:", test_accuracy)

Best Model: RandomForestClassifier(max_depth=14, n_estimators=339)
Best Parameters: {'n_estimators': 339, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 14}
Test Accuracy: 0.8504398826979472


In [87]:
for train_index, val_index in skf.split(X_train.index, Y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[val_index]
    
    # 오버샘플링 적용
    X_train_fold_oversampled, Y_train_fold_oversampled = oversampler.fit_resample(X_train_fold, Y_train_fold)
    
    # 하이퍼파라미터 튜닝 및 모델 학습
    best_model.fit(X_train_fold_oversampled, Y_train_fold_oversampled)
    
    # 모델 평가
    accuracy = best_model.score(X_val_fold, Y_val_fold)
    print("Validation Accuracy:", accuracy)

test_accuracy = best_model.score(X_test, Y_test)
print("Test Accuracy:", test_accuracy)

Validation Accuracy: 0.8382352941176471
Validation Accuracy: 0.8333333333333334
Validation Accuracy: 0.8529411764705882
Validation Accuracy: 0.8676470588235294
Validation Accuracy: 0.8578431372549019
Test Accuracy: 0.8475073313782991


In [96]:
# print(best_model.feature_importances_)

# 중요도 값과 특성 이름을 묶은 튜플 리스트 생성
importance_tuples = list(zip(X_train.columns, best_model.feature_importances_))

# 튜플 리스트를 데이터프레임으로 변환
importance_df = pd.DataFrame(importance_tuples, columns=['Feature', 'Importance'])

# 중요도 기준으로 내림차순 정렬
importance_df = importance_df.sort_values('Importance', ascending=False)

# 결과 출력
print(importance_df)


                    Feature  Importance
13                 OverTime    0.079351
1                 DailyRate    0.067200
11              MonthlyRate    0.060652
6                HourlyRate    0.059569
27        TotalSatisfaction    0.059120
19           YearsAtCompany    0.055750
2          DistanceFromHome    0.054051
14        PercentSalaryHike    0.041018
22     YearsWithCurrManager    0.039202
9                   JobRole    0.038215
20       YearsInCurrentRole    0.036822
12       NumCompaniesWorked    0.035791
23                  Age_cut    0.033828
17    TrainingTimesLastYear    0.033269
21  YearsSinceLastPromotion    0.032975
16         StockOptionLevel    0.032130
18          WorkLifeBalance    0.027403
25         LowMonthlyIncome    0.027075
8                  JobLevel    0.025854
3                 Education    0.025157
4            EducationField    0.025065
10            MaritalStatus    0.024953
7            JobInvolvement    0.022890
0            BusinessTravel    0.020145


In [88]:
for train_index, val_index in skf.split(X_train.index, Y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[val_index]
    
    # 하이퍼파라미터 튜닝 및 모델 학습
    best_model.fit(X_train_fold, Y_train_fold)
    
    # 모델 평가
    accuracy = best_model.score(X_val_fold, Y_val_fold)
    print("Validation Accuracy:", accuracy)

test_accuracy = best_model.score(X_test, Y_test)
print("Test Accuracy:", test_accuracy)

Validation Accuracy: 0.8431372549019608
Validation Accuracy: 0.8627450980392157
Validation Accuracy: 0.8578431372549019
Validation Accuracy: 0.8676470588235294
Validation Accuracy: 0.8627450980392157
Test Accuracy: 0.8533724340175953


In [97]:
# print(best_model.feature_importances_)

# 중요도 값과 특성 이름을 묶은 튜플 리스트 생성
importance_tuples = list(zip(X_train.columns, best_model.feature_importances_))

# 튜플 리스트를 데이터프레임으로 변환
importance_df = pd.DataFrame(importance_tuples, columns=['Feature', 'Importance'])

# 중요도 기준으로 내림차순 정렬
importance_df = importance_df.sort_values('Importance', ascending=False)

# 결과 출력
print(importance_df)


                    Feature  Importance
13                 OverTime    0.079351
1                 DailyRate    0.067200
11              MonthlyRate    0.060652
6                HourlyRate    0.059569
27        TotalSatisfaction    0.059120
19           YearsAtCompany    0.055750
2          DistanceFromHome    0.054051
14        PercentSalaryHike    0.041018
22     YearsWithCurrManager    0.039202
9                   JobRole    0.038215
20       YearsInCurrentRole    0.036822
12       NumCompaniesWorked    0.035791
23                  Age_cut    0.033828
17    TrainingTimesLastYear    0.033269
21  YearsSinceLastPromotion    0.032975
16         StockOptionLevel    0.032130
18          WorkLifeBalance    0.027403
25         LowMonthlyIncome    0.027075
8                  JobLevel    0.025854
3                 Education    0.025157
4            EducationField    0.025065
10            MaritalStatus    0.024953
7            JobInvolvement    0.022890
0            BusinessTravel    0.020145
